# Ordinary Least Squares Regression

### Summary
In this notebook, the ordinary least squares (OLS) method  will be used to estimate all parameters in a linear regression model. All relevant literature refers to this modeling method as "linear regression" and thus will be referred to as such here. 

This model utilizes (blank) features scraped from [basketball-reference.com](https://www.basketball-reference.com/)'s seasons tables ([totals](https://www.basketball-reference.com/leagues/NBA_2019_totals.html) and [advanced](https://www.basketball-reference.com/leagues/NBA_2019_advanced.html)) and predicts the total fantasy points a player will score the following year, otherwise referred to as __future_fantasy_points__ in the dataset. This metric was calculated following data collection according to [Yahoo's ruleset](https://help.yahoo.com/kb/fantasy-basketball/default-league-settings-fantasy-basketball-sln6919.html). 

It's interesting to note that every iteration of a player is considered separately. For example, 2012 LeBron James is a separate record from 2013 LeBron James. This is to say that 2012 LeBron James' __future_fantasy_points__ attribute is the same as his 2013 total fantasy points attribute, or __fantasy_points__ in the pre-processed data.

While around (blank) records were scraped from basketball-reference, only (blank) records exist in the processed dataset. Consider players who were in the league for only one season. These players do not have __future_fantasy_points__ to predict and thus were dropped during the data-processing phase. Of course, the 2018-2019 season was only used to compute __future_fantasy_points__ for 2017-2018 records. See a subset of the data below.

Note: All data used comes from seasons ending in (blank) to 2019.

In [ ]:
import sys
sys.path.append('C:\\Users\\CA015FO\\basketball')
from sqlite import get_data

df = get_data('totals_advanced_nodummies_nonspecific_min41games_normalizedgames_premodel')
df.head()

### Linear Regression
The goal of a linear regression is always the same, in general, and that's to extract a random sample from a population and then to use that random sample to estimate the properties of the population. 

A good question to ask in terms of basketball and this model would be as follows: "By how much does a player's fantasy points next year change for each point scored the year before?" The answer to this question, the weight, is exactly what we're trying to optimize, along with the other (blank) features' weights. 

Below is the linear regression formula, where Y is __future_fantasy_points__ and, in accordance with the example above, $\beta_1$ is the weight attributed to points scored ($x_1$). 

\begin{equation*} Y = \beta_0 + \beta_1 * x_1 + \beta_2 * x_2  +  ...  +  \epsilon \end{equation*}

$\beta_0$, the intercept, and $\epsilon$, the random error, will both be touched on shortly.

### OLS
The name "ordinary least squares" may sound like gibberish to a fresh set of ears, but it actually does a great job of briefly explaining the method's general process. OLS minimizes the sum of the squares of the differences between the observed dependent variable in the dataset and those predicted by the linear function. This is to minimize the differences between the actual __future_fantasy_points__ values and the predicted __future_fantasy_points__ values.

However, the ability to perform minimization doesn't conclude the test as accurate. OLS comes with seven underlying assumptions, six of which must be met by the linear regression in order for the weights to be considered valid.

Most of the assumptions attempt describe the random error, $\epsilon$. Note the word "attempt" here. The random error is truly a random figure that we will never know, as it's a population value that describes the information we don't have.

Thus, in order to assess $\epsilon$ we turn to residuals, or the sample estimate of the error for each observation. Residuals can be calculated as the observed value minus the fitted value, where the observed value in this model is the actual __future_fantasy_points__ figure and the fitted value is the predicted __future_fantasy_points__ figure for a record.

For the remainder of the article, I will go through each OLS assumption one by one and show whether or whether not my model satisfies the assumption.

In [ ]:
import statsmodels.api as sm
x = df.iloc[:,:-1]
x_constant = sm.add_constant(x)
y = df['future_fantasy_points']
model = sm.OLS(y, x_constant).fit()

In [ ]:
# OLS Assumption 1: The regression model is linear in the coefficients and the error term (linearity of the model)
# Observed vs. Predicted Values should be perfectly diagonal with points equally distributed around it
# Residuals vs. Predicted Values should be perfectly flat with points equally distributed around it

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

# def linearity_test(model, y):
fitted_vals = model.predict()
resids = model.resid_pearson

fig, ax = plt.subplots(1,2)

sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color':'red'})
ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
ax[0].set(xlabel='Predicted', ylabel='Observed')

sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color':'red'})
ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
ax[1].set(xlabel='Predicted', ylabel='Standardized Residuals')
ax[1].set_ylim([resids.min()-1, resids.max()+1])

#     return None

# linearity_test(model, y)

In [ ]:
# OLS Assumption 2: The error term has a population mean of zero
# Expectation (mean) of residuals is zero
# Our constant makes sure this is 0
model.resid.mean()

In [ ]:
# No (perfect) multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
vif = [variance_inflation_factor(x_constant.values, i) for i in range(x_constant.shape[1])]
vif_df = pd.DataFrame({'vif':vif[1:]}, index=x.columns)
# vif_df[vif_df['vif'] > 1].sort_values(by='vif', ascending=False)
# vif_df.T['field_goal_percentage']
vif_df.sort_values(by='vif', ascending=False).T

In [ ]:
corr_df = df.corr()
corr_df['total_rebounds'].sort_values(ascending=False)

In [ ]:
model.params['total_rebounds']

In [ ]:
model.pvalues[model.pvalues > 0.05]

In [ ]:
# Homoscedasticity (equal variance) of residuals
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import numpy as np
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def homoscedasticity_test(model):
    fitted_vals = model.predict()
    resids = model.resid
    resids_standardized = model.get_influence().resid_studentized_internal

    fig, ax = plt.subplots(1,2)

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Residuals vs Fitted', fontsize=16)
    ax[0].set(xlabel='Fitted Values', ylabel='Residuals')

    sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Scale-Location', fontsize=16)
    ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

    bp_test = pd.DataFrame(sms.het_breuschpagan(resids, model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plots ----')

homoscedasticity_test(model)

In [ ]:
# No autocorrelation of residuals
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(model.resid, lags=40 , alpha=0.05)
acf.show()

In [ ]:
model.summary()